In [1]:
! pip install -r requirements.txt

In [2]:
# notebooks/load_data.ipynb

import pandas as pd
from db_utils import execute_query, execute_batch_query

# Load the CSV file into a DataFrame
csv_file = 'seed/initial_data_airbnb.csv'
df_listings = pd.read_csv(csv_file)

# Display the first few rows of the DataFrame
df_listings.head()



,Unnamed: 0,id,host_id,host_since,host_is_superhost,location,latitude,longitude,property_type,room_type,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,entire_home_apt,Rating category,average_ratings
0,0,607435380788232654,430149575,2021-11-02,False,North Park Hill,39.760390,-104.92968,Private room in home,Private room,...,2.50,3.00,3.50,4.00,3.00,False,0.11,False,Low rating,3.083333
1,1,545714833502855511,169214047,2018-01-22,False,Hale,39.727850,-104.93783,Entire rental unit,Entire home/apt,...,0.00,0.00,0.00,0.00,0.00,False,0.00,True,No ratings,0.000000
2,2,52429527,107279139,2016-12-14,True,Five Points,39.758520,-104.98846,Entire townhouse,Entire home/apt,...,4.62,4.78,4.78,4.93,4.59,False,2.52,True,High rating,4.795000
3,3,632494576047532593,416194740,2021-07-31,True,West Colfax,39.736019,-105.05072,Entire townhouse,Entire home/apt,...,0.00,0.00,0.00,0.00,0.00,True,0.00,True,No ratings,0.000000
4,4,687768125161080215,133612752,2017-06-05,False,Sunnyside,39.771430,-105.02028,Entire home,Entire home/apt,...,5.00,5.00,5.00,4.92,4.92,False,0.99,True,High rating,4.986667


In [3]:
# Rename location to neighborhood_name
df_listings = df_listings.rename(columns={'location': 'neighborhood_name'})


# Extract unique locations
df_locations = df_listings[['neighborhood_name', 'latitude',
                            'longitude']].drop_duplicates().reset_index(drop=True)
df_locations['location_id'] = df_locations.index + 1

# Extract unique hosts
df_hosts = df_listings[['host_id', 'host_since',
                        'host_is_superhost']].drop_duplicates().reset_index(drop=True)


# Cleanup: `Rating Category`` should be rating_category
df_listings = df_listings.rename(
    columns={'Rating category': 'rating_category'})

# On df_listings, add foreign key for location_id based off of neighborhood_name
df_listings = pd.merge(df_listings, df_locations[['neighborhood_name', 'location_id']],
                       on='neighborhood_name', how='left')

# Change id to listing_id for clarity and consistency
df_listings = df_listings.rename(columns={'id': 'listing_id'})

# Data normalization:  column \"first_review\" is of type date but expression is of type double precision\
df_listings['first_review'] = pd.to_datetime(df_listings['first_review'])
df_listings['last_review'] = pd.to_datetime(df_listings['last_review'])

# Replace NaT with None
df_listings['first_review'] = df_listings['first_review'].replace({
                                                                  pd.NaT: None})
df_listings['last_review'] = df_listings['last_review'].replace({pd.NaT: None})

In [ ]:
# SQL queries to create tables
create_locations_table = """
CREATE TABLE IF NOT EXISTS locations (
    location_id SERIAL PRIMARY KEY,
    neighborhood_name TEXT,
    latitude FLOAT,
    longitude FLOAT
);
"""

create_hosts_table = """
CREATE TABLE IF NOT EXISTS hosts (
    host_id BIGINT PRIMARY KEY,
    host_since DATE,
    host_is_superhost BOOLEAN
);
"""

create_listings_table = """
CREATE TABLE IF NOT EXISTS listings (
    listing_id BIGINT PRIMARY KEY,
    host_id BIGINT REFERENCES hosts(host_id),
    location_id INT REFERENCES locations(location_id),
    property_type TEXT,
    room_type TEXT,
    guest_count INT,
    bathrooms FLOAT,
    beds FLOAT,
    price FLOAT,
    minimum_nights INT,
    maximum_nights INT,
    availability_30 INT,
    availability_60 INT,
    availability_90 INT,
    availability_365 INT,
    number_of_reviews INT,
    number_of_reviews_ltm INT,
    number_of_reviews_l30d INT,
    first_review DATE,
    last_review DATE,
    review_scores_rating FLOAT,
    review_scores_accuracy FLOAT,
    review_scores_cleanliness FLOAT,
    review_scores_checkin FLOAT,
    review_scores_communication FLOAT,
    review_scores_location FLOAT,
    review_scores_value FLOAT,
    instant_bookable BOOLEAN,
    reviews_per_month FLOAT,
    entire_home_apt BOOLEAN,
    rating_category TEXT,
    average_ratings FLOAT
);
"""

# Execute the queries
execute_query(create_locations_table)
execute_query(create_hosts_table)
execute_query(create_listings_table)

In [7]:
# Insert data into locations table
location_data = []
for _, row in df_locations.iterrows():
    location_data.append((row['location_id'], row['neighborhood_name'],
                         row['latitude'], row['longitude'], row['neighborhood_name']))

insert_location = """
INSERT INTO locations (location_id, neighborhood_name, latitude, longitude)
SELECT %s, %s, %s, %s
WHERE NOT EXISTS (
  SELECT 1 FROM locations WHERE neighborhood_name = %s
);
"""
execute_batch_query(insert_location, location_data)

# Insert data into hosts table
host_data = []
for _, row in df_hosts.iterrows():
    host_data.append(
        (row['host_id'], row['host_since'], row['host_is_superhost']))

insert_host = """
INSERT INTO hosts (host_id, host_since, host_is_superhost)
VALUES (%s, %s, %s)
ON CONFLICT (host_id) DO NOTHING;
"""
execute_batch_query(insert_host, host_data)

# Insert data into listings table
listing_data = []
for _, row in df_listings.iterrows():
    listing_data.append((
        row['listing_id'], row['host_id'], row['location_id'], row['property_type'], row['room_type'], row['guest_count'],
        row['bathrooms'], row['beds'], row['price'], row['minimum_nights'], row['maximum_nights'], row['availability_30'],
        row['availability_60'], row['availability_90'], row['availability_365'], row['number_of_reviews'],
        row['number_of_reviews_ltm'], row['number_of_reviews_l30d'], row['first_review'], row['last_review'],
        row['review_scores_rating'], row['review_scores_accuracy'], row['review_scores_cleanliness'],
        row['review_scores_checkin'], row['review_scores_communication'], row['review_scores_location'],
        row['review_scores_value'], row['instant_bookable'], row['reviews_per_month'], row['entire_home_apt'],
        row['rating_category'], row['average_ratings']
    ))

insert_listing = """
INSERT INTO listings (
  listing_id, host_id, location_id, property_type, room_type, guest_count,
  bathrooms, beds, price, minimum_nights, maximum_nights, availability_30,
  availability_60, availability_90, availability_365, number_of_reviews,
  number_of_reviews_ltm, number_of_reviews_l30d, first_review, last_review,
  review_scores_rating, review_scores_accuracy, review_scores_cleanliness,
  review_scores_checkin, review_scores_communication, review_scores_location,
  review_scores_value, instant_bookable, reviews_per_month, entire_home_apt,
  rating_category, average_ratings
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
ON CONFLICT (listing_id) DO NOTHING;
"""

In [8]:
execute_batch_query(insert_listing, listing_data)
